In [1]:
'''
functions for resistance modeling
'''
import import_ipynb

import numpy as np
from scipy.stats import truncnorm
from log_normal_truncated_ab import log_normal_truncated_ab_sample

def GompertzMutation(N, t, K, r_sen ,r_res , r_per, p_mutation):
    # implementation of transitions probabilities and Gompertz growh (no TKI cell kill)
    if sum(N) == 0: #if all compartmetns = 0, set derivitative to 0 to avoid dividing by 0
        dN_sensitive=0
        dN_resistance=0
        dN_persister=0
    else:
        dN_sensitive = r_sen*(np.log(K/(N[0]+N[1]+N[2]))*N[0])    
        if N[1]==0:
            dN_resistance   = p_mutation*r_per*np.log(K/(N[0]+N[1]+N[2]))*N[2]  #growth and transform possibility 
        else:
            dN_resistance   = r_res*np.log(K/(N[0]+N[1]+N[2]))*N[1] + p_mutation*r_per*np.log(K/(N[0]+N[1]+N[2]))*N[2]  #growth and transform possibility 
        dN_persister    = r_per*np.log(K/(N[0]+N[1]+N[2]))*N[2] - p_mutation*r_per*np.log(K/(N[0]+N[1]+N[2]))*N[2]   #transform possibility of persister to resistance
    
    return [dN_sensitive,dN_resistance,dN_persister]

def EGFRkineticsGompertz_opt2(N, t, K, r_sen ,r_res , r_per, p_mutation, beta, drugExposure):
    # implementation of TKI cell kill, transitions probabilities, and Gompertz growh 
    if sum(N) == 0: #if all compartmetns = 0, set derivitative to 0 to avoid dividing by 0
        dN_sensitive=0
        dN_resistance=0
        dN_persister=0
    else:
        dN_sensitive = (1 - beta*drugExposure)*r_sen*(np.log(K/(N[0]+N[1]+N[2]))*N[0])    
        if N[1]==0:
            dN_resistance   = p_mutation*r_per*np.log(K/(N[0]+N[1]+N[2]))*N[2]  #growth and transform possibility 
        else:
            dN_resistance   = r_res*np.log(K/(N[0]+N[1]+N[2]))*N[1] + p_mutation*r_per*np.log(K/(N[0]+N[1]+N[2]))*N[2]  #growth and transform possibility 
        dN_persister    = r_per*np.log(K/(N[0]+N[1]+N[2]))*N[2] - p_mutation*r_per*np.log(K/(N[0]+N[1]+N[2]))*N[2]   #transform possibility of persister to resistance
    return [dN_sensitive,dN_resistance,dN_persister]

def Pharmacokinetics(erlotinib, t,doseinterval):
    #Erlotinib pharmacokinetics from FOO 2012
    #A (d) = 1.0182 + 0.0114 d, where d is the oral dose.
    k = 0.0465*2.4; #k[d] = 0.0465 hours for nonsmokers from Foo 2012
    dErlotinibDaily =  doseinterval - k * erlotinib
    return dErlotinibDaily

def EGFRkineticsGompertz_opt2_Pharmacokinetics(N, t, K, r_sen ,r_res , r_per, p_mutation, beta, doseinterval):
    #Erlotinib pharmacokinetics from FOO 2012
    #A (d) = 1.0182 + 0.0114 d, where d is the oral dose.
    k = 0.0465*2.4; #k[d] = 0.0465 hours for nonsmokers from Foo 2012
    dErlotinibDaily = 10*(doseinterval - k * N[3])
    
    # implementation of TKI cell kill, transitions probabilities, and Gompertz growh 
    if sum(N[0:3]) == 0: #if all compartmetns = 0, set derivitative to 0 to avoid dividing by 0
        dN_sensitive=0
        dN_resistance=0
        dN_persister=0
    else:
        dN_sensitive = (1 - beta*N[3])*r_sen*(np.log(K/(N[0]+N[1]+N[2]))*N[0])    
        if N[1]==0:
            dN_resistance   = p_mutation*r_per*np.log(K/(N[0]+N[1]+N[2]))*N[2]  #growth and transform possibility 
        else:
            dN_resistance   = r_res*np.log(K/(N[0]+N[1]+N[2]))*N[1] + p_mutation*r_per*np.log(K/(N[0]+N[1]+N[2]))*N[2]  #growth and transform possibility 
        dN_persister    = r_per*np.log(K/(N[0]+N[1]+N[2]))*N[2] - p_mutation*r_per*np.log(K/(N[0]+N[1]+N[2]))*N[2]   #transform possibility of persister to resistance
    return [dN_sensitive,dN_resistance,dN_persister,dErlotinibDaily]

def radiation(alpha,beta,d):
    # analytic radiation linear quadratic kill term applied during CRT (assumed alpha beta ratio of 10, d-dose)
    return np.exp(-(alpha*d+beta*d*d))
     
def chemo(xs,c):
    # analytic chemo cell kill term applied during CRT
    return np.exp(-xs*c); #c : concetration
 
def Gettruncnorm(mu,sig,lower,upper,num):
    # truncated normal sampling function for slection of model parameters
    # utilizes functions from from https://people.sc.fsu.edu/~jburkardt/py_src/py_src.html
    X = truncnorm((lower - mu) / sig, (upper - mu) / sig, loc=mu, scale=sig)
    G1=X.rvs(num);  
    return G1

def Gettrunclognorm(mu,sig,lower,upper,num):
    # truncated log normal sampling function for slection of initial tumor size
    # utilizes functions from from https://people.sc.fsu.edu/~jburkardt/py_src/py_src.html
    seed=(np.random.uniform()+1)*10000
    G1=np.zeros(num);
    for i in range(0,num):
        G1[i],seed=log_normal_truncated_ab_sample(mu,sig,lower,upper,seed);
    return G1
 
def Calc_Cumulative_Curve(TimeSeriesEvents):
    ###################################  
    # Calc % Remaining at Each Month ##
    ################################### 
    num_months=60
    distributionEvents = []
    distributionEvents,tbin=np.histogram(TimeSeriesEvents,bins=num_months+1,range=[0,num_months],density=1);
    distributionEvents=[(sum(distributionEvents[i:]))*(tbin[1]-tbin[0]) for i in range(np.size(distributionEvents))]
    Cumulative_Events=np.array(distributionEvents);
    return Cumulative_Events

importing Jupyter notebook from log_normal_truncated_ab.ipynb
